<a href="https://colab.research.google.com/github/harshc1004/Task1/blob/main/Task1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import cv2
import os
import numpy as np
from fpdf import FPDF

def main(video_path, output_dir, train_number='12309'):
    # Create output directory
    os.makedirs(output_dir, exist_ok=True)

    # Load video
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error opening video file")
        return

    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')

    # Background subtractor with adjusted parameters
    subtractor = cv2.createBackgroundSubtractorMOG2(history=200, varThreshold=30, detectShadows=True)

    coach_count = 0
    in_coach = False
    frame_count = 0
    coach_frames = []
    writer = None
    selected_images = []

    # Tunable parameters (adjust based on video)
    gap_threshold = 0.1  # Increased for potentially noisier backgrounds
    min_coach_frames = 50  # Increased to ensure valid coach length
    strip_width = width // 10  # Wider strip for better gap detection

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1

        # Apply background subtraction
        fgmask = subtractor.apply(frame)

        # Preprocess mask to reduce noise
        fgmask = cv2.medianBlur(fgmask, 5)
        _, fgmask = cv2.threshold(fgmask, 127, 255, cv2.THRESH_BINARY)

        # Check for gap: Analyze a central vertical strip
        strip_start = width // 2 - strip_width // 2
        strip_end = width // 2 + strip_width // 2
        strip_mask = fgmask[:, strip_start:strip_end]
        foreground_ratio = np.mean(strip_mask > 0)

        if foreground_ratio > gap_threshold:
            if not in_coach:
                in_coach = True
                coach_count += 1
                coach_dir = os.path.join(output_dir, f"{train_number}_{coach_count}")
                os.makedirs(coach_dir, exist_ok=True)
                video_path_out = os.path.join(coach_dir, f"{train_number}_{coach_count}.mp4")
                writer = cv2.VideoWriter(video_path_out, fourcc, fps, (width, height))
                coach_frames = []
                print(f"Starting coach {coach_count}")

            writer.write(frame)
            coach_frames.append(frame)
        else:
            if in_coach and len(coach_frames) >= min_coach_frames:
                in_coach = False
                writer.release()
                print(f"Ended coach {coach_count} with {len(coach_frames)} frames")

                # Extract frames (every 10th)
                for i, coach_frame in enumerate(coach_frames[::10]):
                    frame_path = os.path.join(coach_dir, f"{train_number}_{coach_count}_{i+1}.jpg")
                    cv2.imwrite(frame_path, coach_frame)

                # Select middle frame
                mid_frame = coach_frames[len(coach_frames)//2]
                mid_frame_path = os.path.join(coach_dir, f"{train_number}_{coach_count}_mid.jpg")
                cv2.imwrite(mid_frame_path, mid_frame)

                # Detect components
                annotated_path = detect_components(mid_frame_path, coach_dir, train_number, coach_count)
                selected_images.append(annotated_path)

    cap.release()
    if writer:
        writer.release()

    print(f"Total coaches detected: {coach_count}")
    generate_report(selected_images, output_dir, train_number, coach_count)

def detect_components(image_path, coach_dir, train_number, coach_count):
    img = cv2.imread(image_path)
    if img is None:
        print(f"Error loading image: {image_path}")
        return image_path

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (7, 7), 0)  # Larger kernel for smoother edges
    edges = cv2.Canny(blurred, 30, 100)  # Adjusted thresholds for better edge detection

    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    annotated_img = img.copy()
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if 3000 < area < 10000:  # Narrower area range for doors
            x, y, w, h = cv2.boundingRect(cnt)
            aspect_ratio = w / float(h)
            if 0.7 < aspect_ratio < 1.5:  # Tighter aspect ratio for door-like shapes
                roi = gray[y:y+h, x:x+w]
                avg_intensity = np.mean(roi)
                label = "Door Open" if avg_intensity < 80 else "Door Closed"  # Adjusted threshold

                cv2.rectangle(annotated_img, (x, y), (x+w, y+h), (0, 255, 0), 2)
                cv2.putText(annotated_img, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    annotated_path = os.path.join(coach_dir, f"{train_number}_{coach_count}_annotated.jpg")
    cv2.imwrite(annotated_path, annotated_img)
    return annotated_path

def generate_report(image_paths, output_dir, train_number, coach_count):
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=12)
    pdf.cell(200, 10, txt=f"Train {train_number} Coverage Report - {coach_count} Coaches", ln=1, align='C')

    for i, img_path in enumerate(image_paths):
        pdf.cell(200, 10, txt=f"Coach {i+1}", ln=1)
        if os.path.exists(img_path):
            pdf.image(img_path, x=10, w=180)
        else:
            pdf.cell(200, 10, txt=f"Image {img_path} not found", ln=1)

    report_path = os.path.join(output_dir, f"{train_number}_report.pdf")
    pdf.output(report_path)
    print(f"Report generated at: {report_path}")

if __name__ == "__main__":
    # For Colab, set video path directly
    video_path = "/content/DHN-side-view-lower-2025-08-31-11-24-27-47.mp4"
    output_dir = "processed_output"
    main(video_path, output_dir)

Starting coach 1
Ended coach 1 with 339 frames
Starting coach 2
Ended coach 2 with 92 frames
Starting coach 3
Ended coach 3 with 56 frames
Starting coach 4
Ended coach 4 with 54 frames
Starting coach 5
Ended coach 5 with 50 frames
Starting coach 6
Ended coach 6 with 53 frames
Starting coach 7
Ended coach 7 with 52 frames
Starting coach 8
Ended coach 8 with 66 frames
Starting coach 9
Ended coach 9 with 52 frames
Starting coach 10
Ended coach 10 with 58 frames
Starting coach 11
Ended coach 11 with 50 frames
Starting coach 12
Ended coach 12 with 53 frames
Starting coach 13
Ended coach 13 with 51 frames
Starting coach 14
Ended coach 14 with 59 frames
Starting coach 15
Ended coach 15 with 50 frames
Starting coach 16
Ended coach 16 with 58 frames
Starting coach 17
Ended coach 17 with 50 frames
Starting coach 18
Ended coach 18 with 61 frames
Starting coach 19
Ended coach 19 with 51 frames
Starting coach 20
Ended coach 20 with 50 frames
Starting coach 21
Ended coach 21 with 55 frames
Starting 